# Legendre transformation and Gibbs energy

In order to enable fatigue load scenarios, this notebook shows how to transform a model derived based on the Helmholtz free energy to the Gibbs free energy 

Given the Helmholtz free energy controlled by strain/displacement $u$ and temperature $\vartheta$
$$
F = \hat{F}(u, \vartheta; \mathcal{E})
$$
we seek the complementary form of the state representaiton that can is expressed using the stress level $\sigma$ instead of the strain $u$   
$$
G = \hat{G}(\sigma, \vartheta; \mathcal{E})
$$
The Legendre transform exploits the fact that 
$$
\hat{F}(u, \vartheta; \mathcal{E})
 + \hat{G}(\sigma, \vartheta; \mathcal{E}) = \sigma u. 
$$

The generic derivation below exploits the fact that stress level definition as the rate of change of $F$ due to change of strain is obtained as the gradient
$$
\sigma = \partial_u F = \Sigma(u) 
$$
which can be symbolically inverted to obtain
$$
u = \Sigma^{-1}(\sigma).
$$

Thus the substitution into the Legendre transform renders
$$
\hat{G}(\sigma, \vartheta; \mathcal{E}) = \sigma \Sigma^{-1}(\sigma) - \hat{F}(\Sigma^{-1}(\sigma), \vartheta; \mathcal{E})
$$

In the framework below, two instances of the `GSM` class are constructed. The first is based on Helmholtz energy and the second on the Gibbs energy. The dissipative potential and internal variables are the same. The only difference is the swapped meaning of the variables `u` and `sig`. In case of Gibbs based GSM, input variable `u` means stress and output variable `sig` means strain. This shall be better formalized in future implementation of the API.

In [ ]:
%matplotlib widget
from bmcs_matmod import GSM
import matplotlib.pylab as plt 
import sympy as sp
from bmcs_utils.api import Cymbol
import numpy as np
sp.init_printing()

## Material parameters

In [ ]:
E_T = Cymbol(r'E_{\mathrm{T}}', codename='E_T_', real=True, nonnegative=True)
gamma_T = Cymbol(r'\gamma_{\mathrm{T}}', codename='gamma_T_', real=True)
K_T = Cymbol(r'K_{\mathrm{T}}', codename='K_T_', real=True)
S_T = Cymbol(r'S_{\mathrm{T}}', codename='S_T_', real=True, nonnegative=True)
r_T = Cymbol(r'r_{\mathrm{T}}', codename='r_T_', real=True, nonnegative=True)
c_T = Cymbol(r'c_{\mathrm{T}}', codename='c_T_', real=True, nonnegative=True)
eta_T = Cymbol(r'\eta_{\mathrm{T}}', codename='eta_T_', real=True, nonnegative=True)
# temperature 
C_v = Cymbol(r'C_{\mathrm{v}}', codename='C_v_', real=True, nonnegative=True)
T_0 = Cymbol(r'\vartheta_0', codename='T_0_', real=True, nonnegative=True)
beta = Cymbol(r'\beta', codename='beta_', real=True, nonnegative=True)

In [ ]:
f_s = Cymbol(r'f_\mathrm{T}', codename='f_s_')

In [ ]:
mparams = (E_T, gamma_T, K_T, S_T, f_s, c_T, r_T, eta_T, C_v, T_0, beta)
mparams

## External state variables

In [ ]:
u_T = Cymbol(r'u_\mathrm{T}', codename='u_T_', real=True)
u_a = sp.Matrix([u_T])
sig_T = Cymbol(r'\sigma_\mathrm{T}', codename='sig_T_', real=True)
sig_a = sp.Matrix([sig_T])
sig_a

In [ ]:
T = Cymbol(r'\vartheta', codename='T_', real=True)
Gamma = sp.exp(-beta * (T - T_0))
Gamma

## Internal state variables

In [ ]:
u_p_T = Cymbol(r'u_\mathrm{T}^\mathrm{p}', codename='u_p_T_', real=True)
u_p_a = sp.Matrix([u_p_T])
sig_p_T = Cymbol(r'\sigma^\mathrm{p}_\mathrm{T}', codename='sig_p_T_', real=True)
sig_p_a = sp.Matrix([sig_p_T])

In [ ]:
omega_T = Cymbol(r'\omega_\mathrm{T}', codename='omega_T_', real=True)
omega_ab = sp.Matrix([[omega_T]])
omega_a = sp.Matrix([omega_T])
Y_T = Cymbol(r'Y_\mathrm{T}', codename='Y_T_', real=True)
Y_a = sp.Matrix([Y_T])

In [ ]:
z_T = Cymbol(r'z_\mathrm{T}', codename='z_T_', real=True, nonnegative=True)
z_a = sp.Matrix([z_T])
K_ab = sp.Matrix([[K_T]])
Z_T = Cymbol(r'Z_\mathrm{T}', codename='Z_T_', real=True, nonnegative=True)
Z_a = sp.Matrix([Z_T])

In [ ]:
alpha_T = Cymbol(r'\alpha_\mathrm{T}', codename='alpha_T_', real=True, nonnegative=True)
gamma_ab = sp.Matrix([[gamma_T]])
alpha_a = sp.Matrix([alpha_T])
X_T = Cymbol(r'X_\mathrm{T}', codename='X_T_', real=True, nonnegative=True)
X_a = sp.Matrix([X_T])

## Free energy potential

In [ ]:
E_ab = sp.Matrix([[E_T]])
u_el_a = u_a - u_p_a
E_eff_ab = (sp.eye(1) - omega_ab) * E_ab
E_eff_ab

In [ ]:
U_e_ = sp.Rational(1,2) * (u_el_a.T * E_eff_ab * u_el_a)[0]
U_p_ = sp.Rational(1,2) * (z_a.T * K_ab * z_a + alpha_a.T * gamma_ab * alpha_a)[0]
TS_ = C_v * (T - T_0) **2 / (2 * T_0)
F_ = U_e_ + U_p_ + - TS_
F_ = U_e_ + U_p_ - TS_
F_

## Dissipation potential

In [ ]:
sig_eff_T = sp.Function(r'\sigma^{\mathrm{eff}}_{\mathrm{T}}')(sig_p_T, omega_T)
q_T = sp.Function(r'q_Tx')(sig_eff_T,X_T)
norm_q_T = sp.sqrt(q_T*q_T)
subs_q_T = {q_T: (sig_eff_T - X_T)}
subs_sig_eff = {sig_eff_T: sig_p_T / (1-omega_T) }
y = Cymbol(r'y')
f_s = Cymbol(r'f_s_')
f_solved_ = sp.sqrt(y**2) - f_s
f_ = (f_solved_
      .subs({y: norm_q_T})
      .subs(subs_q_T)
      .subs(subs_sig_eff)
      .subs(f_s,((f_s  * Gamma + Z_T)))
      )

In [ ]:
f_

In [ ]:
phi_T = (1 - omega_T)**c_T * S_T / (r_T+1) * (Y_T / S_T)**(r_T+1)
phi_ext_ = phi_T
phi_ext_

In [ ]:
t_relax_T_ = eta_T / (E_T + K_T + gamma_T)
t_relax_ = sp.Matrix([
                    t_relax_T_,
                    t_relax_T_,
                    t_relax_T_,
                    ] 
               )

## Gibbs free energy

In [ ]:
dF_du = F_.diff(u_a)
dF_du = dF_du.xreplace({h: 0 for h in dF_du.atoms(sp.DiracDelta)})
dF_du = dF_du.xreplace({h: 1 for h in dF_du.atoms(sp.Heaviside)})
u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
                            zip(sig_a, u_a, dF_du)])
subs_u_sig_ = dict(zip(u_a, u_sig_))

sig_x_u_ = (sig_a.T * u_a)[0]
G_expr = sig_x_u_ - F_
G_ = sp.simplify(G_expr.subs(subs_u_sig_))
G_

In [ ]:
gsm_F = GSM(
    u_vars = u_a,
    sig_vars = sig_a,
    T_var = T,
    m_params = mparams,
    Eps_vars = (u_p_a, z_a, alpha_a, omega_a),
    Sig_vars = (sig_p_a, Z_a, X_a, Y_a),
    Sig_signs = (-1, 1, 1, -1),
    F_expr = F_,
    f_expr = f_,
    phi_ext_expr = phi_ext_,
    t_relax = t_relax_
)

gsm_G = GSM(
    u_vars = sig_a,
    sig_vars = u_a,
    T_var = T,
    m_params = mparams,
    Eps_vars = (u_p_a, z_a, alpha_a, omega_a),
    Sig_vars = (sig_p_a, Z_a, X_a, Y_a),
    Sig_signs = (1, -1, -1, 1),
    F_expr = G_,
    f_expr = f_,
    phi_ext_expr = phi_ext_,
    t_relax = t_relax_
)

In [ ]:
gsm_F.F_expr

In [ ]:
gsm_G.F_expr

In [ ]:
sp.simplify(gsm_F.dF_dEps_), sp.simplify(gsm_G.dF_dEps_)

In [ ]:
gsm_F.Sig_vars, gsm_F.Sig_, sp.simplify(gsm_G.Sig_)

In [ ]:
sp.simplify(gsm_F.df_dSig_), sp.simplify(gsm_G.df_dSig_)

In [ ]:
sp.simplify(gsm_F.Phi_), sp.simplify(gsm_G.Phi_)

In [ ]:
sp.simplify(gsm_F.df_dEps_), sp.simplify(gsm_G.df_dEps_)

In [ ]:
gsm_G.phi_, sp.simplify(gsm_G.Phi_)

In [ ]:
gsm_F.dSig_dEps_, sp.simplify(gsm_G.dSig_dEps_)

In [ ]:
import inspect
source = inspect.getsource(gsm_G._df_dlambda_lambdified)
print(source)

In [ ]:
_u_a = np.array([1.1])
_T = np.array([20])
material_params = dict(
    E_T_=2, gamma_T_=1, K_T_=1, S_T_=1000, c_T_=1, 
    f_s_=1, r_T_=2,
    eta_T_=1, 
    T_0_=20, C_v_=1, beta_=1
)
_Eps_B00 = np.zeros((gsm_F.n_Eps_explicit,), np.float_ )
_f = gsm_F.get_f(_u_a, 20, _Eps_B00, _Eps_B00, **material_params)
_df_dlambda = gsm_F.get_df_dlambda(_u_a, 20, _Eps_B00, _Eps_B00, **material_params)
_Sig = gsm_F.get_Sig(_u_a, 20, _Eps_B00, _Eps_B00, **material_params)
_Eps_B00.shape, _Sig.shape

In [ ]:
_Sig = gsm_F.get_Sig(_u_a, 20, _Eps_B00, _Eps_B00, **material_params)
gsm_F.get_df_dSig(_u_a, 20, _Eps_B00, _Sig, **material_params)

In [ ]:
_df_dlambda = gsm_F.get_df_dlambda(_u_a, 20, _Eps_B00, _Sig, **material_params)

In [ ]:
_f2, _df_dlambda2, _Sig_B00 = gsm_F.get_f_df_Sig(_u_a, _T, _Eps_B00, _Eps_B00, **material_params)

## Monotonic loading

In [ ]:
sig_ = gsm_F.F_expr.diff(gsm_F.u_vars)
sig_

In [ ]:
material_params = dict(
    E_T_=10, gamma_T_=1000, K_T_=10, S_T_=0.14, c_T_=2, f_s_=1, 
    eta_T_=20, r_T_=2,
    T_0_=20, C_v_=1, beta_=1
)

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t = response
    _u_atI, _Eps_btI, _Sig_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI 

gsm_F.vp_on = True
gsm_F.update_at_k = False
gsm_G.vp_on = True
gsm_G.update_at_k = False

In [ ]:
# params
n_t = 151
n_I = 1
u_T_max = 2
t_t = np.linspace(0, 1, n_t)
u_ta_F = (u_T_max * t_t).reshape(-1, 1)
T_t = 20 + t_t * 0
_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F = gsm_run(gsm_F, u_ta_F, T_t, t_t, **material_params)
_max_sig = np.max(_sig_atI_F)
_max_sig

In [ ]:
fig, (ax, ax_T) = plt.subplots(1,2)
ax.plot(_u_atI_F[0, :, 0], _sig_atI_F[0, :, 0]);
ax_T.plot(_t_t_F, _T_t_F);

In [ ]:
# params
n_t = 151
n_I = 1
t_t = np.linspace(0, 1, n_t)
u_ta_G = (_max_sig * t_t).reshape(-1, 1)
T_t = 20 + t_t * 0
_t_t_G, _u_atI_G, _sig_atI_G, T_t, _Eps_btI_G, _Sig_btI_G = gsm_run(gsm_G, u_ta_G, T_t, t_t, **material_params)

## Fatigue loading 

In [ ]:

def generate_cyclic_load(max_s, min_s, freq, total_cycles, points_per_cycle):
    # Calculate the time for one cycle
#    total_cycles = freq * total_time
    total_time = total_cycles / freq

    # Calculate the mean value and amplitude
    mean_value = (max_s + min_s) / 2
    amplitude = (max_s - min_s) / 2

    # Calculate the initial loading slope
    slope = 2 * np.pi * freq * amplitude
    
    # Time arrays for linear increase and sinusoidal part
    initial_duration = mean_value / slope
    initial_points = int(initial_duration * freq * points_per_cycle)
    total_points = int(total_time * freq * points_per_cycle)
    
    # Generate the initial linear increase
    initial_t = np.linspace(0, initial_duration, initial_points, endpoint=False)
    initial_loading = slope * initial_t

    # Generate the sinusoidal loading
    sinusoidal_t = np.linspace(0, total_time, total_points, endpoint=False)
    sinusoidal_loading = mean_value + amplitude * np.sin(2 * np.pi * freq * sinusoidal_t)

    # Combine the initial linear increase with the sinusoidal loading
    t_full = np.concatenate((initial_t, sinusoidal_t + initial_duration))
    s_full = np.concatenate((initial_loading, sinusoidal_loading))
    
    return t_full, s_full

fig, ax = plt.subplots(1,1, figsize=(8,3))
t_t, s_t = generate_cyclic_load(0.66, 0.1, 5, 1000, 20)
ax.plot(t_t, s_t, '-o')

In [ ]:
# params
t_t, s_t = generate_cyclic_load(0.66, 0.1, 5, 1000, 20)
u_ta_fat = (_max_sig * s_t).reshape(-1, 1)
T_t = 20 + t_t * 0
_t_t_fat, _u_atI_fat, _sig_atI_fat, _T_t_fat, _Eps_btI_fat, _Sig_btI_fat = gsm_run(gsm_G, u_ta_fat, T_t, t_t, **material_params)

In [ ]:
def sig_p_T_0_lambdified(gsm_):
    sig_p_T_solved_ = sp.solve(gsm_.f_, sig_p_T)
    return sp.lambdify((gsm_.u_vars, gsm_.T_var, 
                                gsm_.Eps.as_explicit(), 
                                gsm_.Sig.as_explicit()) + gsm_.m_params + ('**kw',), 
                            sig_p_T_solved_, cse=True)
get_sig_p_T_0 = sig_p_T_0_lambdified(gsm_G)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(8,8), tight_layout=True)
fig.canvas.header_visible=False
colors = ['red', 'blue', 'orange', 'gray']

ax1.set_title('stress-strain')
ax1.plot(_u_atI_F[0,:,0], _sig_atI_F[0,:,0], color='black', ls='dashed')
ax1.plot(_sig_atI_G[0,:,0], _u_atI_G[0, :,0], color='black')

ax2.set_title('damage')
ax3.set_title('elastic domain')
ax4.set_title('fatigue creep')

S_max_levels = np.array([0.95, 0.85, 0.75, 0.65])
alpha_line = 0.6
for S_max, c in zip(S_max_levels, colors):

    print('S_max', S_max)
    # params
    t_t, s_t = generate_cyclic_load(S_max, 0.1, 5, 1000, 30)
    u_ta_fat = (_max_sig * s_t).reshape(-1, 1)
    T_t = 20 + t_t * 0
    _t_t_fat, _u_atI_fat, _sig_atI_fat, _T_t_fat, _Eps_btI_fat, _Sig_btI_fat = gsm_run(gsm_G, u_ta_fat, T_t, t_t, **material_params)
    _sig_atI_top, _sig_atI_bot = get_sig_p_T_0(_u_atI_fat, _T_t_fat, _Eps_btI_fat, _Sig_btI_fat, **material_params )

    ax1.plot(_sig_atI_fat[0,:,0], _u_atI_fat[0,:,0], color=c, alpha=alpha_line, label=f'$S_{{\max}} = {S_max}$' )
    ax1.set_xlabel(r'$\varepsilon$/-')
    ax1.set_ylabel(r'$\sigma$/MPa')

    _u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_G.Eps_as_blocks(_Eps_btI_fat)
    ax2.plot(_t_t_fat, _omega_atI[0, :, 0], color=c, alpha=alpha_line)
    ax2.set_xlabel(r'$t$/s')
    ax2.set_ylabel(r'$\omega$/-')

    ax3.plot(_t_t_fat, _sig_atI_top[:, 0], color=c, alpha=alpha_line)
    ax3.plot(_t_t_fat, _sig_atI_bot[:, 0], color=c, alpha=alpha_line)
    ax3.fill_between(_t_t_fat, _sig_atI_bot[:, 0], _sig_atI_top[:, 0], color=c, alpha=0.1)
    ax3.set_xlabel(r'$t$/s')
    ax3.set_ylabel(r'$\sigma$/MPa')

    ax4.plot(_t_t_fat, _sig_atI_fat[0,:,0], color=c, alpha=alpha_line)
    ax4.set_xlabel(r'$t$/s')
    ax4.set_ylabel(r'$\varepsilon$/-')

ax1.legend()
    # ax4_T.plot(_t_t_fat, _T_t_fat, color=c, ls='dashed')

In [ ]:
from pathlib import Path
path = Path().home() / 'simdb' / 'data' / 'S_max_effect_high.pdf'
fig.savefig(path)